In [1]:
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
# !pip install lightgbm

In [3]:
df = pd.read_csv('data_after_preprocessing.csv', index_col=0)
df.head()

,text,label
0,"tuần , báo sggp tổ_chức tọa_đàm “ góp_ý dự_thả...",-1
1,nghị_quyết 18 - nq / tw yêu_cầu quy_định thuế ...,1
2,", đất đánh_thuế . ( ảnh minh_họa : ttxvn ) 16/...",1
3,"quy_định khung giá đất , sở_hữu nhà_đất đánh_t...",1
4,nghị_quyết 18 - nq / tw xây_dựng cơ_sở tổng_kế...,1


In [4]:
df['label'].value_counts()


 1    450
-1    305
Name: label, dtype: int64

In [5]:
X_train, X_test, Y_train, Y_test= model_selection.train_test_split(df[['text']],df['label'],test_size=0.15,random_state=25)

In [6]:
# lb = LabelEncoder()
# lb.fit(X_train['sourceName'])
# le_dict = dict(zip(lb.classes_, lb.transform(lb.classes_)))
# X_train['sourceName'] = X_train['sourceName'].apply(lambda x : le_dict.get(x,-1))
# X_test['sourceName'] = X_test['sourceName'].apply(lambda x : le_dict.get(x,-1))

In [7]:
Tfidf_vect = TfidfVectorizer(max_features=1500)
Tfidf_vect.fit(X_train['text'].to_list())
tfidf_train = Tfidf_vect.transform(X_train['text'])
tfidf_test = Tfidf_vect.transform(X_test['text'])

In [8]:
print(tfidf_train.shape)
# train_source = np.array(X_train['sourceName'])
# train_source = np.array([train_source])
# test_source = np.array(X_test['sourceName'])
# test_source = np.array([test_source])

(641, 1500)


In [9]:
tfidf_train = tfidf_train.toarray()
tfidf_test = tfidf_test.toarray()

In [10]:
# tfidf_train.shape
# train_source.shape

In [11]:
# tfidf_train = np.concatenate((tfidf_train, train_source.T),axis=1)
# tfidf_test = np.concatenate((tfidf_test, test_source.T),axis=1)

In [12]:
# print(Tfidf_vect.vocabulary_)

In [13]:
# param_grid = {  'C': [0.1,1, 10, 100], 
#                 'gamma': ['auto',1,0.1,0.01,0.001],
#                 'kernel': ['rbf', 'poly', 'sigmoid','linear'],
#                 'coef0': [0.0, 0.1,0.2, 0.5, 0.8, 1]
#             }

In [14]:
# grid = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2,scoring = 'roc_auc')
# grid.fit(tfidf_train,Y_train)

In [15]:
# grid.best_params_

In [16]:
SVM = svm.SVC(C=1.0, kernel='linear', gamma='auto', degree = 3)
SVM.fit(tfidf_train,Y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(tfidf_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Y_test)*100)

SVM Accuracy Score ->  82.45614035087719


In [17]:
print(classification_report(Y_test,predictions_SVM))

              precision    recall  f1-score   support

          -1       0.80      0.77      0.79        48
           1       0.84      0.86      0.85        66

    accuracy                           0.82       114
   macro avg       0.82      0.82      0.82       114
weighted avg       0.82      0.82      0.82       114



In [18]:
lgbm = LGBMClassifier()
lgbm.fit(tfidf_train,Y_train)
predictions_lgbm = lgbm.predict(tfidf_test)
# Use accuracy_score function to get the accuracy
print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, Y_test)*100)
print(classification_report(Y_test,predictions_lgbm))

ligbm Accuracy Score ->  79.82456140350878
              precision    recall  f1-score   support

          -1       0.78      0.73      0.75        48
           1       0.81      0.85      0.83        66

    accuracy                           0.80       114
   macro avg       0.79      0.79      0.79       114
weighted avg       0.80      0.80      0.80       114



In [19]:
rf = RandomForestClassifier(n_estimators=1000,min_samples_leaf=2,
                                min_samples_split= 10, max_features = 'sqrt', criterion = 'entropy', bootstrap= True,
                              random_state=42)
rf.fit(tfidf_train,Y_train) 
predictions_rf = rf.predict(tfidf_test)
# Use accuracy_score function to get the accuracy
print("rf Accuracy Score -> ",accuracy_score(predictions_rf, Y_test)*100)
print(classification_report(Y_test,predictions_rf))

rf Accuracy Score ->  82.45614035087719
              precision    recall  f1-score   support

          -1       0.85      0.71      0.77        48
           1       0.81      0.91      0.86        66

    accuracy                           0.82       114
   macro avg       0.83      0.81      0.81       114
weighted avg       0.83      0.82      0.82       114

